In [78]:
f_leftandright = open('leftandright', 'rb')
left_lst = []
right_lst = []
pan_lst = []
tilt_lst = []

for i in range(5120):
  left = f_leftandright.read(4)
  right = f_leftandright.read(4)
  pan = f_leftandright.read(4)
  tilt = f_leftandright.read(4)
  left_lst.append(int.from_bytes(left, byteorder = 'little' ,signed = True))
  right_lst.append(int.from_bytes(right, byteorder = 'little' ,signed = True))
  pan_lst.append(int.from_bytes(pan, byteorder = 'little' ,signed = True))
  tilt_lst.append(int.from_bytes(tilt, byteorder = 'little' ,signed = True))

f_offset = open('offset', 'rb')
panoff_lst = []
tiltoff_lst = []
rotateoff_lst = []
tranoff_lst = []

for i in range(5120):
  panoff = f_offset.read(4)
  tiltoff = f_offset.read(4)
  rotateoff = f_offset.read(4)
  tranoff = f_offset.read(4)
  panoff_lst.append(int.from_bytes(panoff, byteorder = 'little' ,signed = True))
  tiltoff_lst.append(int.from_bytes(tiltoff, byteorder = 'little' ,signed = True))
  rotateoff_lst.append(int.from_bytes(rotateoff, byteorder = 'little' ,signed = True))
  tranoff_lst.append(int.from_bytes(tranoff, byteorder = 'little' ,signed = True))

rotatecom = []
translatecom = []
for i in range(5120):
  rotatecom.append((right_lst[i] - left_lst[i])/2)
  translatecom.append((right_lst[i] + left_lst[i])/2)


In [79]:
import numpy as np

import tensorflow as tf

In [80]:
size = 5120
datanum = 10
purpose_offset = rotateoff_lst
purpose_value = rotatecom

In [81]:
input_lst = []
for i in range(size - datanum + 1):
  box = []
  for j in range(i, i+datanum, 1):
    box.append(purpose_offset[j])
  input_lst.append(box)

label_lst = []
for i in range(datanum - 1, size, 1):
  box = []
  box.append(purpose_value[i])
  label_lst.append(box)

In [82]:
rotateoff_numpy = np.array(input_lst)
rotatecom_numpy = np.array(label_lst)

In [83]:
mean_vals = np.mean(rotateoff_numpy, axis=0)
std_val = np.std(rotateoff_numpy)

mean_vals_com = np.mean(rotatecom_numpy, axis=0)
std_val_com = np.std(rotatecom_numpy)

In [84]:
rotateoff_numpy_centered = (rotateoff_numpy - mean_vals)/std_val
rotatecom_numpy_centered = (rotatecom_numpy - mean_vals_com)/std_val_com

# rotateoff_numpy_centered = (rotateoff_numpy - np.max(rotateoff_numpy))/(np.max(rotateoff_numpy) - np.min(rotateoff_numpy))
# rotatecom_numpy_centered = (rotatecom_numpy - np.max(rotatecom_numpy))/(np.max(rotatecom_numpy) - np.min(rotatecom_numpy))

In [85]:
train_x = np.ones((4000, datanum))
train_y = np.ones((4000, 1))
test_x = np.ones((1120 - datanum + 1, datanum))
test_y = np.ones((1120 - datanum + 1, 1))

for i in range(4000):
  train_x[i] = rotateoff_numpy_centered[i]
  train_y[i] = rotatecom_numpy_centered[i]

for i in range(4000, 5120 - datanum + 1, 1):
  test_x[i - 4000] = rotateoff_numpy_centered[i]
  test_y[i - 4000] = rotatecom_numpy_centered[i]

In [86]:
print(test_x)

[[-0.6966206  -0.78573578 -0.44449874 ... -0.27210525 -0.61837298
  -0.66787561]
 [-0.78565739 -0.44442809 -0.17244188 ... -0.61835943 -0.66783787
  -0.65798263]
 [-0.4443497  -0.17237123 -0.56321446 ... -0.66782432 -0.65794489
  -1.11800605]
 ...
 [-1.10717913 -1.13693646  1.50441767 ...  0.50943991  0.84084108
  -0.19301273]
 [-1.13685806  1.50448832  1.08396616 ...  0.84085463 -0.19297499
  -0.52937394]
 [ 1.50456671  1.08403681  0.7673909  ... -0.19296144 -0.52933619
  -0.68766156]]


In [87]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Dense(
        units=50,    
        input_dim = train_x.shape[1],
        kernel_initializer='glorot_normal',
        bias_initializer='zeros',
        activation='tanh'))
model.add(tf.keras.layers.Dropout(.2, input_shape=(2,)))
model.add(
    tf.keras.layers.Dense(
        units=100,    
        input_dim=100,
        kernel_initializer='glorot_normal',
        bias_initializer='zeros',
        activation='tanh'))
model.add(tf.keras.layers.Dropout(.2, input_shape=(2,)))
model.add(
    tf.keras.layers.Dense(
        units=train_y.shape[1],    
        input_dim=100,
        kernel_initializer='glorot_normal',
        bias_initializer='zeros',
        activation=None))

In [88]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 50)                550       
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 5,751
Trainable params: 5,751
Non-trainable params: 0
_________________________________________________________________


In [89]:
sgd_optimizer = tf.keras.optimizers.SGD(
    lr=0.000001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,
              loss='mse')

In [ ]:
history = model.fit(train_x, train_y,
                    batch_size=1, epochs=50,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
3600/3600 [==============================] - 4s 1ms/step - loss: 0.6863 - val_loss: 0.2523
Epoch 2/50
3600/3600 [==============================] - 4s 1ms/step - loss: 0.3091 - val_loss: 0.1437
Epoch 3/50
3600/3600 [==============================] - 4s 1ms/step - loss: 0.2309 - val_loss: 0.1159
Epoch 4/50
3600/3600 [==============================] - 4s 1ms/step - loss: 0.2204 - val_loss: 0.1016
Epoch 5/50
3600/3600 [==============================] - 4s 1ms/step - loss: 0.2058 - val_loss: 0.0926
Epoch 6/50
1438/3600 [==========>...................] - ETA: 2s - loss: 0.1735

In [ ]:
import matplotlib.pyplot as plt

loss_and_metrics = model.evaluate(test_x, test_y, batch_size=1)
xhat = test_x
yhat = model.predict(xhat)
print(yhat)


SSE_lst = [] 
SST_lst = []

means = test_y.sum()/test_y.shape[0]

for i in range(xhat.shape[0]):
  SSE_lst.append((test_y[i] - yhat[i])**2)
  SST_lst.append((test_y[i] - means)**2)

R_square = 1 - (sum(SSE_lst) / sum(SST_lst))


plt.plot(test_x, test_y, '.', color = 'b') 
plt.plot(xhat, yhat, '.', color = 'r') 

plt.show()

print(R_square)


In [ ]:
plt.plot(test_y, yhat, '.', color = 'b') 


plt.show()
